In [ ]:
import tensorflow as tf
#tf.test.gpu_device_name()
from google.colab import drive

# 使用工具colab的接口挂载google drive目录，这样可以从外部获取数据并且可以把训练好的模型保存在google drive上
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip3 install SpeechRecognition
!pip3 install wheel
!sudo apt-get install python python-all-dev python-pip build-essential swig git libpulse-dev libasound2-dev
!pip3 install pocketsphinx
!pip3 install -U sentence-transformers
!pip3 install jiwer
from jiwer import wer

     |████████████████████████████████| 32.8 MB 147 kB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
python is already the newest version (2.7.15~rc1-1).
git is already the newest version (1:2.17.1-1ubuntu0.9).
libasound2-dev is already the newest version (1.1.3-5ubuntu0.6).
The following additional packages will be installed:
  libpulse-mainloop-glib0 libpython-all-dev python-all python-asn1crypto
  python-cffi-backend python-crypto python-cryptography python-dbus
  python-enum34 python-gi python-idna python-ipaddress python-keyring
  python-keyrings.alt python-pip-whl python-pkg-resources python-secretstorage
  python-setuptools python-six python-wheel python-xdg swig3.0
Suggested packages:
  python-crypto-doc python-cryptography-doc python-cryptography-vectors
  python-dbus-dbg python-dbus-doc python-enum34-doc python-gi-cairo
  gnome-keyring libkf5wallet-bin gir1.2-gnom

变量初始化,先用google试一试

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')
#model = SentenceTransformer('all-MiniLM-L12-v2')
#倒是也行，但是感觉少了一半

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#get bert embedding
#from bert_embedding import BertEmbedding
from sklearn.metrics.pairwise import cosine_similarity
def get_bert(new_translation):
  #bert_embedding = BertEmbedding(),
  test=[new_translation]
  test_text_embed = model.encode(test)
  return test_text_embed

def bert_simi(ori_bert, new_bert):
  return cosine_similarity(ori_bert, new_bert)


In [ ]:
import speech_recognition as speech

import speech_recognition as speech

def transcribe_google(audiopath):

  recognizer = speech.Recognizer()
  harvard = speech.AudioFile(audiopath)  # 读取音频文件
  with harvard as source:
      audio = recognizer.record(source)
  # 使用google识别语音
  try:
      #print("google识别结果为:" + recognizer.recognize_google(audio))
      #换一下sphinx
      return recognizer.recognize_google(audio)
  except speech.UnknownValueError:
      pass
  except speech.RequestError as e:
      pass

In [ ]:
#暂时用一下，后面肯定要再改
import scipy.io.wavfile as wav
import copy
def store_new(flag,fs,x,path_store):
  data_final=copy.deepcopy(x)
  for i in range(len(flag)):
    if(flag[i]==0):
      data_final[512*i:512*(i+1)]=copy.deepcopy(512*[0])
  
  wav.write(path_store, fs, data_final)
  
  



In [ ]:
from sys import flags
import scipy.io.wavfile as wav
import pandas as pd
import os
import wave
import numpy as np
import copy
import random
#add a function to compare simialrity and get a true and flase and one score
def compare_tfidf(key, new_translation):

#比较tf-idf的时候需要的是新的一句翻译，然后对它进行分词，得到一个分词列表，然后通过key去查阅对应的字典及其概率，然后相加，得到正确或者错误以及extent
#需要：key,分词算法，translation
#return: boolean 和一个score
  if new_translation== None:
    return False, 0.0 
  dic_=np.load('/content/gdrive/MyDrive/new_deepcover/translation/commonvoice_tf_idf_normalized.npy',allow_pickle=True)
  tfidf_all=dic_.item()
  tfidf_ori=tfidf_all[key]
  word=new_translation.strip().split()
  words=list(set(word))
  score=0
  for i in range (len(words)):
    if(words[i] in tfidf_ori.keys()):
      score+=tfidf_ori[words[i]]

  if(score>0.5):
    score=(score-0.5)/0.5
    return True, score
  else:
    score=(0.5-score)/0.5
    return False, score 


def compare_bert(key, new_translation):
  if new_translation== None:
    return False, 0.0 
  dic_=np.load('/content/gdrive/MyDrive/new_deepcover/translation/bert_commonvoice.npy',allow_pickle=True)
  bert_all=dic_.item()
  ori_bert=bert_all[key]
  new_bert=get_bert(new_translation)
  score=bert_simi(ori_bert, new_bert)[0][0]
  print("========================================")
  print(score)
  if(score>0.5):
    score=(score-0.5)/0.5
    return True, score
  else:
    score=(0.5-score)/0.5
    return False, score


#需要：key,bertembeding算法，translation
#return: boolean 和一个score


In [ ]:
from re import T
import scipy.io.wavfile as wav
import pandas as pd
import os
import wave
import numpy as np
import copy
import random
import math

def get_mutations_bert(path_x,path_store,y,key,mutation_size,mutation_factor):

  
  #it should be in a function
  #存变种，标号对应的是帧的标号，原版全1
  mutation={}
  #存变种的Y
  mutation_y={}
  #存n个帧的四个指标，后续计算他们的四个SFLmeasures，写在下面

  failing=[]
  passing=[]

  #failure extent and success extent
  fail_extent={}
  success_extent={}


  #number of mutations and mutation_factor should be passed in
  mutation_size=mutation_size
  mutation_factor=mutation_factor

  #number of frames,path of audio and path to restore something these things should be passed in
  path_x=path_x
  path_store=path_store
  y=y

  #compute number of frames
  fs, x = wav.read(path_x)
  print(len(x))
  n_f=(len(x)-1536)//512
  index_=[i for i in range(n_f)]
  flag_x=[1]*n_f
  portion=int(n_f*mutation_factor)
  #number of frames to be selected and number to finetune to make sure the balance between failing and passing
  incr=math.floor(1/3*portion)
  if portion<1: portion=1
  if incr<1:incr=1
  i=0
  print("portion:")
  print(portion)


  #for i in range(mutation_size):
  while(len(passing)+len(failing)<mutation_size):
    print("======================================================================================")
    print(i)
    print("failing:")
    print(failing)
    print("passing:")
    print(passing)
    index_random=random.sample(index_,portion)
    flag=[0 if index in index_random else value for index, value in enumerate(flag_x)]
    print("flag")
    print(flag)
    mutation[i]=flag
    #call speech recognize to get the y, needs to be chosen and code needs to be filled up.
    path_temp=path_store+"/"+key+"_temp.wav"
    store_new(flag,fs,x,path_temp)
    mutation_y[i]=transcribe_google(path_temp)
    print(mutation_y[i])
    print(mutation_y)
    #compare的两个函数
    still_right, score=compare_bert(key,mutation_y[i])
    #需要修改，要修改如何认定失败以及到底有多失败，要存好权重
    if(still_right!=True):
      is_adv=True
    else:
      is_adv=False

    #based on the is_adv, factor needs to be changed,there is a bug, it is actually a global thing也可以每次返回自循环
    if is_adv:
      print("fail!!!")
      failing.append(i)#add this mutation in failing part
      fail_extent[i]=score
      print(fail_extent)
      i+=1
      ite=portion #testgen_factor
      while ite>1: #ite>0.01:
        #ite=ite-1#ite//2 #ite=(ite+0)/2
        ite=int(ite-incr)
        if ite<1: break
        index_temp=random.sample(index_,ite)

        flag_temp=[0 if index in index_temp else value for index, value in enumerate(flag_x)]
        #there needs code
        path_temp=path_store+"/"+key+"_temp.wav"
        store_new(flag_temp,fs,x,path_temp)
        new_y=transcribe_google(path_temp)
        still_right, score=compare_bert(key,new_y)
        if (still_right):
          print("===========================zheshi1bianxiaozhaodaode==============suceess")
          print(i)
          print(flag_temp)
          mutation[i]=flag_temp
          mutation_y[i]=new_y
          print(mutation_y)
          passing.append(i)
          success_extent[i]=score
          print(success_extent)
          i+=1
          break
    else:
      print("passing")
      passing.append(i)#add this mutation in failing part
      success_extent[i]=score
      print(success_extent)
      i+=1
      ite=portion #testgen_factor
      while ite<n_f/2: #ite>0.01:
        #ite=ite-1#ite//2 #ite=(ite+0)/2
        ite=int(ite+incr)
        if ite>n_f/2: break
        index_temp=random.sample(index_,ite)

        flag_temp=[0 if index in index_temp else value for index, value in enumerate(flag_x)]
        #there needs code
        path_temp=path_store+"/"+key+"_temp.wav"
        store_new(flag_temp,fs,x,path_temp)
        new_y=transcribe_google(path_temp)
        still_right, score=compare_bert(key,new_y)
        if (still_right==False):
          print("===========================zheshi1bianxiaozhaodaode==============fail")
          print(i)
          print(flag_temp)
          mutation[i]=flag_temp
          mutation_y[i]=new_y
          print(mutation_y)
          failing.append(i)
          fail_extent[i]=score
          print(fail_extent)
          i+=1
          break
  return mutation,mutation_y,flag_x,fail_extent,success_extent

def get_rank(mutation, mutation_y,flag_x,fail_extent,success_extent,metric):

  #需要计算那几个矩阵然后计算指标之后排序啦
  ef=np.zeros((len(flag_x),),dtype=float)
  nf=np.zeros((len(flag_x),),dtype=float)
  ep=np.zeros((len(flag_x),),dtype=float)
  np_=np.zeros((len(flag_x),),dtype=float)

  for i in range(len(mutation)):
    #我觉得这里，执行条件的意思对应的就是将某个东西mask掉，被mask相当于被执行
    diffs=np.array(flag_x)-np.array(mutation[i])
    #diffs=np.array(mutation[i])-np.array(flag_x)
    if i in fail_extent.keys():
      is_adv=True
    else:
      is_adv=False
    #is_adv=(mutation_y[i]!=y)
    ds_i1=diffs.copy()
    ds_i1[ds_i1>0]=1
    ds_i2=diffs.copy()
    ds_i2[ds_i2>0]=-1
    ds_i2[ds_i2==0]=+1
    ds_i2[ds_i2==-1]=0
    if is_adv:
      #这里需要注意一下形状是否真的一致，感觉这里有点不对劲，感觉得看看定义。那几个矩阵，我这里进行了修改，但是不确定, 要乘上extent
      #然后就是换了下位置，因为被masked算执行
      ef=ef+ds_i1*fail_extent[i]
      nf=nf+ds_i2*fail_extent[i]
    else:
      ep=ep+ds_i1*success_extent[i]
      np_=np_+ds_i2*success_extent[i]


    

  #要计算那几个指标，整理的时候需要分函数 metrics needed be passed
  print(ef)
  print(nf)
  if metric=='random':
    #spectrum=np.random.rand(sp[0], sp[1], sp[2])
    print("this metric doesnt exist.")

  elif metric=='zoltar':
    zoltar=np.zeros((len(flag_x),), dtype=float)
    for index, x in np.ndenumerate(np.array(flag_x)):
      aef=ef[index]
      anf=nf[index]
      anp=np_[index]
      aep=ep[index]
      if aef==0:
        zoltar[index]=0
      else:
        k=(10000.0*anf*aep)/aef
        zoltar[index]=(aef*1.0)/(aef+anf+aep+k)
    score=zoltar
  elif metric=='wong-ii':
    wong=np.zeros((len(flag_x),), dtype=float)
    for index, x in np.ndenumerate(np.array(flag_x)):
      aef=ef[index]
      anf=nf[index]
      anp=np_[index]
      aep=ep[index]
      wong[index]=aef-aep
    score=wong
  elif metric=='ochiai':
    ochiai=np.zeros((len(flag_x),), dtype=float)
    for index, x in np.ndenumerate(np.array(flag_x)):
      aef=ef[index]
      anf=nf[index]
      anp=np_[index]
      aep=ep[index]
      try:
        ochiai[index]=aef/np.sqrt((aef+anf)*(aef+aep))
      except: ochiai[index]=0
    score=ochiai
  elif metric=='tarantula':
    tarantula=np.zeros((len(flag_x),), dtype=float)
    for index, x in np.ndenumerate(np.array(flag_x)):
      aef=ef[index]
      anf=nf[index]
      anp=np_[index]
      aep=ep[index]
      try: tarantula[index]=(aef/(aef+anf))/(aef/(aef+anf)+anp/(aep+anp))
      except: tarantula[index]=0
    score=tarantula
  else:
    raise Exception('The measure is not supported: {0}'.format(metric))


  print (np.argsort(-score)) # 逆序输出索引，从大到小
  rank=np.argsort(-score)
  return rank


#迭代rank数组，建造新音频看看到啥时候停下,这是后续的，我先看看能不能跑
'''
newaudio_x=[0]*len(x)
newaudio_x[512*n_f:]=copy.deepcopy(x[512*n_f+1:])
for x in np.nditer(rank):
  newaudio_x[512*x,512*(x+1)]=copy.deepcopy(x[512*x,512*(x+1)])
  newaudio_y=
  #get the new y
  if(newaudio_y!=y):
    continue
  else:
    break

print("the newaudio is found!")
'''









'\nnewaudio_x=[0]*len(x)\nnewaudio_x[512*n_f:]=copy.deepcopy(x[512*n_f+1:])\nfor x in np.nditer(rank):\n  newaudio_x[512*x,512*(x+1)]=copy.deepcopy(x[512*x,512*(x+1)])\n  newaudio_y=\n  #get the new y\n  if(newaudio_y!=y):\n    continue\n  else:\n    break\n\nprint("the newaudio is found!")\n'

In [ ]:
#tf-idf
from re import T
import scipy.io.wavfile as wav
import pandas as pd
import os
import wave
import numpy as np
import copy
import random
import math

def get_mutations_tfidf(path_x,path_store,y,key,mutation_size,mutation_factor):

  
  #it should be in a function
  #存变种，标号对应的是帧的标号，原版全1
  mutation={}
  #存变种的Y
  mutation_y={}
  #存n个帧的四个指标，后续计算他们的四个SFLmeasures，写在下面

  failing=[]
  passing=[]

  #failure extent and success extent
  fail_extent={}
  success_extent={}


  #number of mutations and mutation_factor should be passed in
  mutation_size=mutation_size
  mutation_factor=mutation_factor

  #number of frames,path of audio and path to restore something these things should be passed in
  path_x=path_x
  path_store=path_store
  y=y

  #compute number of frames
  fs, x = wav.read(path_x)
  print(len(x))
  n_f=(len(x)-1536)//512
  index_=[i for i in range(n_f)]
  flag_x=[1]*n_f
  portion=int(n_f*mutation_factor)
  #number of frames to be selected and number to finetune to make sure the balance between failing and passing
  incr=math.floor(1/3*portion)
  if portion<1: portion=1
  if incr<1:incr=1
  i=0
  print("portion:")
  print(portion)


  #for i in range(mutation_size):
  while(len(passing)+len(failing)<mutation_size):
    print("======================================================================================")
    print(i)
    print("failing:")
    print(failing)
    print("passing:")
    print(passing)
    index_random=random.sample(index_,portion)
    flag=[0 if index in index_random else value for index, value in enumerate(flag_x)]
    print("flag")
    print(flag)
    mutation[i]=flag
    #call speech recognize to get the y, needs to be chosen and code needs to be filled up.
    path_temp=path_store+"/"+key+"_temp.wav"
    store_new(flag,fs,x,path_temp)
    mutation_y[i]=transcribe_google(path_temp)
    print(mutation_y[i])
    print(mutation_y)
    #compare的两个函数
    still_right, score=compare_tfidf(key,mutation_y[i])
    #需要修改，要修改如何认定失败以及到底有多失败，要存好权重
    if(still_right!=True):
      is_adv=True
    else:
      is_adv=False

    #based on the is_adv, factor needs to be changed,there is a bug, it is actually a global thing也可以每次返回自循环
    if is_adv:
      print("fail!!!")
      failing.append(i)#add this mutation in failing part
      fail_extent[i]=score
      print(fail_extent)
      i+=1
      ite=portion #testgen_factor
      while ite>1: #ite>0.01:
        #ite=ite-1#ite//2 #ite=(ite+0)/2
        ite=int(ite-incr)
        if ite<1: break
        index_temp=random.sample(index_,ite)

        flag_temp=[0 if index in index_temp else value for index, value in enumerate(flag_x)]
        #there needs code
        path_temp=path_store+"/"+key+"_temp.wav"
        store_new(flag_temp,fs,x,path_temp)
        new_y=transcribe_google(path_temp)
        still_right, score=compare_tfidf(key,new_y)
        if (still_right):
          print("===========================zheshi1bianxiaozhaodaode==============suceess")
          print(i)
          print(flag_temp)
          mutation[i]=flag_temp
          mutation_y[i]=new_y
          print(mutation_y)
          passing.append(i)
          success_extent[i]=score
          print(success_extent)
          i+=1
          break
    else:
      print("passing")
      passing.append(i)#add this mutation in failing part
      success_extent[i]=score
      print(success_extent)
      i+=1
      ite=portion #testgen_factor
      while ite<n_f/2: #ite>0.01:
        #ite=ite-1#ite//2 #ite=(ite+0)/2
        ite=int(ite+incr)
        if ite>n_f/2: break
        index_temp=random.sample(index_,ite)

        flag_temp=[0 if index in index_temp else value for index, value in enumerate(flag_x)]
        #there needs code
        path_temp=path_store+"/"+key+"_temp.wav"
        store_new(flag_temp,fs,x,path_temp)
        new_y=transcribe_google(path_temp)
        still_right, score=compare_tfidf(key,new_y)
        if (still_right==False):
          print("===========================zheshi1bianxiaozhaodaode==============fail")
          print(i)
          print(flag_temp)
          mutation[i]=flag_temp
          mutation_y[i]=new_y
          print(mutation_y)
          failing.append(i)
          fail_extent[i]=score
          print(fail_extent)
          i+=1
          break
  return mutation,mutation_y,flag_x,fail_extent,success_extent

def get_rank(mutation, mutation_y,flag_x,fail_extent,success_extent,metric):

  #需要计算那几个矩阵然后计算指标之后排序啦
  ef=np.zeros((len(flag_x),),dtype=float)
  nf=np.zeros((len(flag_x),),dtype=float)
  ep=np.zeros((len(flag_x),),dtype=float)
  np_=np.zeros((len(flag_x),),dtype=float)

  for i in range(len(mutation)):
    #我觉得这里，执行条件的意思对应的就是将某个东西mask掉，被mask相当于被执行
    diffs=np.array(flag_x)-np.array(mutation[i])
    #diffs=np.array(mutation[i])-np.array(flag_x)
    if i in fail_extent.keys():
      is_adv=True
    else:
      is_adv=False
    #is_adv=(mutation_y[i]!=y)
    ds_i1=diffs.copy()
    ds_i1[ds_i1>0]=1
    ds_i2=diffs.copy()
    ds_i2[ds_i2>0]=-1
    ds_i2[ds_i2==0]=+1
    ds_i2[ds_i2==-1]=0
    if is_adv:
      #这里需要注意一下形状是否真的一致，感觉这里有点不对劲，感觉得看看定义。那几个矩阵，我这里进行了修改，但是不确定, 要乘上extent
      #然后就是换了下位置，因为被masked算执行
      ef=ef+ds_i1*fail_extent[i]
      nf=nf+ds_i2*fail_extent[i]
    else:
      ep=ep+ds_i1*success_extent[i]
      np_=np_+ds_i2*success_extent[i]


    

  #要计算那几个指标，整理的时候需要分函数 metrics needed be passed
  print(ef)
  print(nf)
  if metric=='random':
    #spectrum=np.random.rand(sp[0], sp[1], sp[2])
    print("this metric doesnt exist.")

  elif metric=='zoltar':
    zoltar=np.zeros((len(flag_x),), dtype=float)
    for index, x in np.ndenumerate(np.array(flag_x)):
      aef=ef[index]
      anf=nf[index]
      anp=np_[index]
      aep=ep[index]
      if aef==0:
        zoltar[index]=0
      else:
        k=(10000.0*anf*aep)/aef
        zoltar[index]=(aef*1.0)/(aef+anf+aep+k)
    score=zoltar
  elif metric=='wong-ii':
    wong=np.zeros((len(flag_x),), dtype=float)
    for index, x in np.ndenumerate(np.array(flag_x)):
      aef=ef[index]
      anf=nf[index]
      anp=np_[index]
      aep=ep[index]
      wong[index]=aef-aep
    score=wong
  elif metric=='ochiai':
    ochiai=np.zeros((len(flag_x),), dtype=float)
    for index, x in np.ndenumerate(np.array(flag_x)):
      aef=ef[index]
      anf=nf[index]
      anp=np_[index]
      aep=ep[index]
      try:
        ochiai[index]=aef/np.sqrt((aef+anf)*(aef+aep))
      except: ochiai[index]=0
    score=ochiai
  elif metric=='tarantula':
    tarantula=np.zeros((len(flag_x),), dtype=float)
    for index, x in np.ndenumerate(np.array(flag_x)):
      aef=ef[index]
      anf=nf[index]
      anp=np_[index]
      aep=ep[index]
      try: tarantula[index]=(aef/(aef+anf))/(aef/(aef+anf)+anp/(aep+anp))
      except: tarantula[index]=0
    score=tarantula
  else:
    raise Exception('The measure is not supported: {0}'.format(metric))


  print (np.argsort(-score)) # 逆序输出索引，从大到小
  rank=np.argsort(-score)
  return rank


#迭代rank数组，建造新音频看看到啥时候停下,这是后续的，我先看看能不能跑
'''
newaudio_x=[0]*len(x)
newaudio_x[512*n_f:]=copy.deepcopy(x[512*n_f+1:])
for x in np.nditer(rank):
  newaudio_x[512*x,512*(x+1)]=copy.deepcopy(x[512*x,512*(x+1)])
  newaudio_y=
  #get the new y
  if(newaudio_y!=y):
    continue
  else:
    break

print("the newaudio is found!")
'''









'\nnewaudio_x=[0]*len(x)\nnewaudio_x[512*n_f:]=copy.deepcopy(x[512*n_f+1:])\nfor x in np.nditer(rank):\n  newaudio_x[512*x,512*(x+1)]=copy.deepcopy(x[512*x,512*(x+1)])\n  newaudio_y=\n  #get the new y\n  if(newaudio_y!=y):\n    continue\n  else:\n    break\n\nprint("the newaudio is found!")\n'

In [ ]:
import random

def get_mutations_im(path_x,path_store,y,key,mutation_size,mutation_factor):

  
  #it should be in a function
  #存变种，标号对应的是帧的标号，原版全1
  mutation={}
  #存变种的Y
  mutation_y={}
  #存n个帧的四个指标，后续计算他们的四个SFLmeasures，写在下面

  failing=[]
  passing=[]

  #failure extent and success extent
  fail_extent={}
  success_extent={}


  #number of mutations and mutation_factor should be passed in
  mutation_size=mutation_size
  mutation_factor=mutation_factor

  #number of frames,path of audio and path to restore something these things should be passed in
  path_x=path_x
  path_store=path_store
  y=y

  #compute number of frames
  fs, x = wav.read(path_x)
  print(len(x))
  n_f=(len(x)-1536)//512
  index_=[i for i in range(n_f)]
  flag_x=[1]*n_f
  portion=int(n_f*mutation_factor)
  #number of frames to be selected and number to finetune to make sure the balance between failing and passing
  incr=math.floor(1/3*portion)
  if portion<1: portion=1
  if incr<1:incr=1
  i=0
  print("portion:")
  print(portion)


  #for i in range(mutation_size):
  while(len(passing)+len(failing)<mutation_size):
    print("======================================================================================")
    print(i)
    print("failing:")
    print(failing)
    print("passing:")
    print(passing)
    index_random=random.sample(index_,portion)
    flag=[0 if index in index_random else value for index, value in enumerate(flag_x)]
    print("flag")
    print(flag)
    mutation[i]=flag
    #call speech recognize to get the y, needs to be chosen and code needs to be filled up.
    path_temp=path_store+"/"+key+"_temp.wav"
    store_new(flag,fs,x,path_temp)
    mutation_y[i]=transcribe_google(path_temp)
    print(mutation_y[i])
    print(mutation_y)
    #compare的两个函数
    if(mutation_y[i]==None):
      continue

    still_right=wer(y,mutation_y[i])
    #score=1
    #需要修改，要修改如何认定失败以及到底有多失败，要存好权重
    if(still_right!=0.0):
      is_adv=True
      score=wer(y,mutation_y[i])
    else:
      is_adv=False
      score=1.0

    #based on the is_adv, factor needs to be changed,there is a bug, it is actually a global thing也可以每次返回自循环
    if is_adv:
      print("fail!!!")
      failing.append(i)#add this mutation in failing part
      fail_extent[i]=score
      print(fail_extent)
      i+=1
      ite=portion #testgen_factor
      while ite>1: #ite>0.01:
        #ite=ite-1#ite//2 #ite=(ite+0)/2
        ite=int(ite-incr)
        if ite<1: break
        index_temp=random.sample(index_,ite)

        flag_temp=[0 if index in index_temp else value for index, value in enumerate(flag_x)]
        #there needs code
        path_temp=path_store+"/"+key+"_temp.wav"
        store_new(flag_temp,fs,x,path_temp)
        new_y=transcribe_google(path_temp)
        if(new_y==None):
          continue
        still_right=wer(y,new_y)


        if (still_right==0.0):
          print("===========================zheshi1bianxiaozhaodaode==============suceess")
          print(i)
          print(flag_temp)
          mutation[i]=flag_temp
          mutation_y[i]=new_y
          print(mutation_y)
          passing.append(i)
          success_extent[i]=1.0
          print(success_extent)
          i+=1
          break
    else:
      print("passing")
      passing.append(i)#add this mutation in failing part
      success_extent[i]=score
      print(success_extent)
      i+=1
      ite=portion #testgen_factor
      while ite<n_f/2: #ite>0.01:
        #ite=ite-1#ite//2 #ite=(ite+0)/2
        ite=int(ite+incr)
        if ite>n_f/2: break
        index_temp=random.sample(index_,ite)

        flag_temp=[0 if index in index_temp else value for index, value in enumerate(flag_x)]
        #there needs code
        path_temp=path_store+"/"+key+"_temp.wav"
        store_new(flag_temp,fs,x,path_temp)
        new_y=transcribe_google(path_temp)
        if(new_y==None):
          continue
        still_right=wer(y,new_y)
        if (still_right!=0.0):
          print("===========================zheshi1bianxiaozhaodaode==============fail")
          print(i)
          print(flag_temp)
          mutation[i]=flag_temp
          mutation_y[i]=new_y
          print(mutation_y)
          failing.append(i)
          fail_extent[i]=still_right
          print(fail_extent)
          i+=1
          break
  return mutation,mutation_y,flag_x,fail_extent,success_extent

def get_rank(mutation, mutation_y,flag_x,fail_extent,success_extent,metric):

  #需要计算那几个矩阵然后计算指标之后排序啦
  ef=np.zeros((len(flag_x),),dtype=float)
  nf=np.zeros((len(flag_x),),dtype=float)
  ep=np.zeros((len(flag_x),),dtype=float)
  np_=np.zeros((len(flag_x),),dtype=float)

  for i in range(len(mutation)):
    #我觉得这里，执行条件的意思对应的就是将某个东西mask掉，被mask相当于被执行
    diffs=np.array(flag_x)-np.array(mutation[i])
    #diffs=np.array(mutation[i])-np.array(flag_x)
    if i in fail_extent.keys():
      is_adv=True
    else:
      is_adv=False
    #is_adv=(mutation_y[i]!=y)
    ds_i1=diffs.copy()
    ds_i1[ds_i1>0]=1
    ds_i2=diffs.copy()
    ds_i2[ds_i2>0]=-1
    ds_i2[ds_i2==0]=+1
    ds_i2[ds_i2==-1]=0
    if is_adv:
      #这里需要注意一下形状是否真的一致，感觉这里有点不对劲，感觉得看看定义。那几个矩阵，我这里进行了修改，但是不确定, 要乘上extent
      #然后就是换了下位置，因为被masked算执行
      ef=ef+ds_i1*fail_extent[i]
      nf=nf+ds_i2*fail_extent[i]
    else:
      ep=ep+ds_i1*success_extent[i]
      np_=np_+ds_i2*success_extent[i]


    

  #要计算那几个指标，整理的时候需要分函数 metrics needed be passed
  print(ef)
  print(nf)
  if metric=='random':
    #spectrum=np.random.rand(sp[0], sp[1], sp[2])
    print("this metric doesnt exist.")

  elif metric=='zoltar':
    zoltar=np.zeros((len(flag_x),), dtype=float)
    for index, x in np.ndenumerate(np.array(flag_x)):
      aef=ef[index]
      anf=nf[index]
      anp=np_[index]
      aep=ep[index]
      if aef==0:
        zoltar[index]=0
      else:
        k=(10000.0*anf*aep)/aef
        zoltar[index]=(aef*1.0)/(aef+anf+aep+k)
    score=zoltar
  elif metric=='wong-ii':
    wong=np.zeros((len(flag_x),), dtype=float)
    for index, x in np.ndenumerate(np.array(flag_x)):
      aef=ef[index]
      anf=nf[index]
      anp=np_[index]
      aep=ep[index]
      wong[index]=aef-aep
    score=wong
  elif metric=='ochiai':
    ochiai=np.zeros((len(flag_x),), dtype=float)
    for index, x in np.ndenumerate(np.array(flag_x)):
      aef=ef[index]
      anf=nf[index]
      anp=np_[index]
      aep=ep[index]
      try:
        ochiai[index]=aef/np.sqrt((aef+anf)*(aef+aep))
      except: ochiai[index]=0
    score=ochiai
  elif metric=='tarantula':
    tarantula=np.zeros((len(flag_x),), dtype=float)
    for index, x in np.ndenumerate(np.array(flag_x)):
      aef=ef[index]
      anf=nf[index]
      anp=np_[index]
      aep=ep[index]
      try: tarantula[index]=(aef/(aef+anf))/(aef/(aef+anf)+anp/(aep+anp))
      except: tarantula[index]=0
    score=tarantula
  else:
    raise Exception('The measure is not supported: {0}'.format(metric))


  print (np.argsort(-score)) # 逆序输出索引，从大到小
  rank=np.argsort(-score)
  return rank

In [ ]:
def merge_two_dicts(x, y):
    """Given two dicts, merge them into a new dict as a shallow copy."""
    z = x.copy()
    z.update(y)
    return z

#zhge主要是因为要把每次产生的rank合并，因为不合并很难一次搞出来

In [ ]:
path_x="sample.wav"

#key=key
y=transcribe_google(path_x)
path_store="path_you_wish_to_place_the_mutants"
metric='tarantula'
print(key)

mutation,mutation_y,flag_x,fail_extent,success_extent=get_mutations_tfidf(path_x,path_store,y,key,mutation_size=100,mutation_factor=0.05)
rank=get_rank(mutation,mutation_y,flag_x,fail_extent,success_extent,metric='tarantula')
rank_list_tfidf[key]=rank
print(rank)
#new_=merge(two_dics(rank_all_tfidf,rank_list_tfidf))
np.save("path_of_rank",rank_list_tfidf)
print("done")

组建新音频